In [ ]:
import tensorflow as tf
print(tf.__version__)

2.11.0


# Import data

In [ ]:
import tensorflow_datasets as tfds
imdb,info = tfds.load('imdb_reviews',with_info=True,as_supervised=True)

# Splitting the data

In [ ]:
import numpy as np 
train_data,test_data = imdb['train'], imdb['test']

In [ ]:
train_data

<PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
training_sentences = []
training_labels  = []

testing_sentences = []
testing_labels = []

# Loop over all training exaxmples and save the sentences and labels
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

# Loop over all testing examples and save the senteces and labels
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

In [ ]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

# Tokenization

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
truc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length,truncating=truc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

# Creating the Model

In [ ]:
model = tf.keras.Sequential([
    # tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

In [ ]:
len(sequences[0])

117

In [ ]:
len(padded),len(training_labels)

(25000, 25000)

In [ ]:
# Fit the model
model.fit(x=padded,
          y=training_labels_final,
          epochs = 10,
          validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 6s 6ms/step - loss: 0.5562 - accuracy: 0.7435 - val_loss: 0.3969 - val_accuracy: 0.8339
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3358 - accuracy: 0.8620 - val_loss: 0.3300 - val_accuracy: 0.8592
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2729 - accuracy: 0.8906 - val_loss: 0.3283 - val_accuracy: 0.8567
Epoch 4/10
782/782 [==============================] - 4s 6ms/step - loss: 0.2370 - accuracy: 0.9066 - val_loss: 0.3386 - val_accuracy: 0.8560
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2108 - accuracy: 0.9211 - val_loss: 0.3617 - val_accuracy: 0.8476
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1901 - accuracy: 0.9311 - val_loss: 0.3905 - val_accuracy: 0.8427
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1718 - accuracy: 0.9377 - val_loss: 0.4158 - val_accuracy: 0.8391
Epoch 

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [ ]:
reverse_word_index = tokenizer.index_word

In [ ]:
# Download the Files
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

In [ ]:
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for word_num in range(1,vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word+'\n')
  out_v.write('\t'.join([str(x) for x in embeddings])+'\n')
out_v.close()
out_m.close()

**Experiment with Below**

https://projector.tensorflow.org/